In [5]:
# imports
import requests
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
latandlongs = pd.Series()
latandlongs = station_stats['latitude'].astype(str) + ',' + station_stats['longitude'].astype(str)
FSak = os.environ["FOURSQUARE_API_KEY"]
FSurl = "https://api.foursquare.com/v3/places/search?"
FSheaders = {
    "Accept": "application/json",
    "Authorization": FSak
}

In [ ]:
def get_POI_df(latandlong, categories):
    FSparams = {
      	"ll": latandlong,
        "radius": "1000",
        "categories": categories,
        "fields": "fsq_id,categories,distance,name,social_media,verified,rating,stats,location,popularity",
      	"sort":"DISTANCE", 
        "limit" :"50"
    }
    FSresponse = requests.get(FSurl, params=FSparams, headers=FSheaders)
    FSjson= FSresponse.json()
    data = pd.json_normalize(FSjson["results"])
    return data
    

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# this DF will contain All unique POIs for all stations
poi_for_all_stations = pd.DataFrame()
# this DF will contain data about bike stations and the corresponding POI (distance from each other)
station_to_poi_table = pd.DataFrame(columns = ['station_id', 'fsq_id', 'distance']) 
for indx1,latandlong in latandlongs.items():
    tempdata = get_POI_df([latandlong],"13065")
    poi_for_all_stations = pd.concat([poi_for_all_stations, tempdata], ignore_index=True)
    for indx2,row in tempdata.iterrows():
        #indx1 indicates row of the station, indx2 signifies the row of the FOURSQUARE place within the df for each station
        new_row = pd.DataFrame({'station_id': [(station_stats.loc[indx1,"id"])],'fsq_id' :[(tempdata.loc[indx2,"fsq_id"])],
                               'distance':[(tempdata.loc[indx2,"distance"])]})
        station_to_poi_table = pd.concat([station_to_poi_table,new_row],ignore_index=True)
        
# further cleaning of data_frame
poi_for_all_stations.drop_duplicates(subset = ["fsq_id"],inplace = True, ignore_index = True)
poi_for_all_stations.drop(columns=['distance'], inplace = True) 



Put your parsed results into a DataFrame

In [ ]:
# I completed this task above

Creating addtitional dataframe to record poi and category, and dataframe for cat_id and corresponding category name

In [ ]:
category_df = poi_for_all_stations.loc[:,["fsq_id","categories"]]
cat_table = pd.DataFrame(columns = ['Index', 'Category_id'])

for indx, values in category_df.loc[:,"categories"].items():
    for value in values:
        new_row = {'Index': indx, 'Category_id':value["id"]}
       
        cat_table = pd.concat([cat_table, pd.DataFrame([new_row])], ignore_index=True)

cat_table = pd.merge(category_df, cat_table, right_on= 'Index',
                   left_index = True , 
                   how = 'inner')

def get_cat_details(df):
    get_details_table = pd.DataFrame(columns = ['cat_name', 'cat_id'])
    for indx, row in df.iterrows():
        for indx2 in row[1]:
            new_row = pd.DataFrame({'cat_name': [indx2["name"]],'cat_id':[indx2["id"]]})
            get_details_table= pd.concat([get_details_table,new_row],ignore_index=True)
        #print(row)
    return(get_details_table)

cat_details_table = get_cat_details(cat_table)
cat_details_table = cat_details_table.drop_duplicates().reset_index(drop = True)

cat_table = cat_table.drop(columns=['Index',"categories"])

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
Yak = os.environ["YELP_API_KEY"]
Yurl = "https://api.yelp.com/v3/businesses/search?"
Yheaders = {
    "Accept": "application/json",
    "Authorization": 'Bearer %s' % Yak
}
def get_yelp_poi(lat, long):
    Yparams = {
        "latitude" : lat,
        "longitude" : long,
        "radius" : "1000",
        "categories" : "restaurants",
        "sort_by" : "distance",
        "limit" : "50"
    }
    Yresponse = requests.get(Yurl,params=Yparams, headers = Yheaders)
    Yjson = Yresponse.json()
    ydata = pd.json_normalize(Yjson["businesses"])
    return ydata

yelp_all_poi = pd.DataFrame()
for i in range(len(station_stats)):
    lat = station_stats.loc[i, "latitude"]
    long = station_stats.loc[i, "longitude"]
    ytemp = get_yelp_poi(lat, long)
    yelp_all_poi = pd.concat([yelp_all_poi, ytemp], ignore_index=True)

yelp_all_poi.drop_duplicates(subset = ["id"],inplace = True, ignore_index = True)



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# I completed this task above

Put your parsed results into a DataFrame

In [ ]:
# I completed this task above

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Each API has it's own distinctive edge (ex, Foursquare offers data like social media handles, YELP offers filtering businesses with desired reservation time and number of people). Foursquare's API had more POIs, YELP's API had more number of reviews per POI. But the deciding factor for me was a measurment named 'popularity' that measures visits and the distance between POI and bike station which only foursquare's api offered. 

Get the top 10 restaurants according to their rating

In [ ]:
top_yelp =(yelp_all_poi.sort_values(by='rating', ascending=False, ignore_index= True).head(10)).loc[:,["name"]] # yelp top 10
top_FS = (poi_for_all_stations.sort_values(by='rating', ascending=False, ignore_index= True).head(10)).loc[:,["name"]] # fs top 10